In [6]:
pip install shap

Note: you may need to restart the kernel to use updated packages.


In [7]:
%%writefile stock_predictor.py

import numpy as np
import pandas as pd
import yfinance as yf
from keras.models import load_model
import streamlit as st
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import shap

# Load the model
model = load_model('C:\\Users\\User\\Stock Prediction Model.keras')

# Streamlit header
st.header('Stock Market Predictor')

# User input for stock symbol
stock = st.text_input('Enter Stock Symbol', 'GOOG')
start = '2012-01-01'
end = '2025-08-11'

# Download stock data
data = yf.download(stock, start, end)

# Display stock data
st.subheader('Stock Data')
st.write(data)

# Prepare the training and test datasets
data_train = pd.DataFrame(data.Close[0: int(len(data) * 0.80)])
data_test = pd.DataFrame(data.Close[int(len(data) * 0.80):])

# Scaling the data
scaler = MinMaxScaler(feature_range=(0, 1))
pas_100_days = data_train.tail(100)
data_test = pd.concat([pas_100_days, data_test], ignore_index=True)
data_test_scale = scaler.fit_transform(data_test)

# Plot Price vs MA50
st.subheader('Price vs MA50')
ma_50_days = data.Close.rolling(50).mean()
fig1 = plt.figure(figsize=(8, 6))
plt.plot(ma_50_days, 'r', label='MA50')
plt.plot(data.Close, 'g', label='Closing Price')
plt.legend()
st.pyplot(fig1)

# Plot Price vs MA50 vs MA100
st.subheader('Price vs MA50 vs MA100')
ma_100_days = data.Close.rolling(100).mean()
fig2 = plt.figure(figsize=(8, 6))
plt.plot(ma_50_days, 'r', label='MA50')
plt.plot(ma_100_days, 'b', label='MA100')
plt.plot(data.Close, 'g', label='Closing Price')
plt.legend()
st.pyplot(fig2)

# Plot Price vs MA100 vs MA200
st.subheader('Price vs MA100 vs MA200')
ma_200_days = data.Close.rolling(200).mean()
fig3 = plt.figure(figsize=(8, 6))
plt.plot(ma_100_days, 'r', label='MA100')
plt.plot(ma_200_days, 'b', label='MA200')
plt.plot(data.Close, 'g', label='Closing Price')
plt.legend()
st.pyplot(fig3)

# Prepare the data for prediction
x = []
y = []

for i in range(100, data_test_scale.shape[0]):
    x.append(data_test_scale[i-100:i])
    y.append(data_test_scale[i, 0])

x, y = np.array(x), np.array(y)

# Predict the stock prices
predict = model.predict(x)

# Inverse scale the predicted and actual prices
scale = 1 / scaler.scale_
predict = predict * scale
y = y * scale

# Plot Original Price vs Predicted Price
st.subheader('Original Price vs Predicted Price')
fig4 = plt.figure(figsize=(10, 8))
plt.plot(data.index[-len(predict):],predict, color='red', linestyle='dashed', label='Predicted Price')
plt.plot(data.index[-len(y):],y, color='g', label='Original Price')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
st.pyplot(fig4)


# Convert data to NumPy array if not already
data_test_np = np.array(data_test_scale)

# Create a SHAP explainer (choose the appropriate one for your model)
# Example with shap.Explainer (recommended for general use)
explainer = shap.Explainer(model, data_test_np)

# Calculate SHAP values for the last observation
shap_values = explainer(data_test_np[-1].reshape(1, -1))

# Decision-making based on predictions

# Get the last predicted price and the last actual price
last_predicted_price = predict[-1]
last_actual_price = y[-1]

# Threshold for deciding Buy/Sell/Hold
buy_threshold = 0.02  # 2% increase
sell_threshold = 0.02  # 2% decrease

# Calculate percentage difference
price_diff_percentage = (last_predicted_price - last_actual_price) / last_actual_price
price_diff_percentage = float(price_diff_percentage)

# Decision logic with detailed explanation
if price_diff_percentage > buy_threshold:
    decision = "Buy"
    explanation = (f"The model predicts a significant price increase of {price_diff_percentage:.2%}.\n"
                  f"Recent trend analysis indicates that the stock has been performing well, with the price "
                  f"expected to continue its upward trajectory.\n"
                  "Key factors contributing to this prediction include strong market momentum and positive "
                  "influences from technical indicators such as the Moving Averages (MA50 and MA100).\n"
                  "Given the current market conditions and this positive outlook, this could be an opportune "
                  "moment to invest in the stock.\n"
                  "However, consider external factors such as upcoming earnings reports and broader economic trends before making a final decision.")
elif price_diff_percentage < -sell_threshold:
    decision = "Sell"
    explanation = (f"The model predicts a notable price decrease of {price_diff_percentage:.2%}.\n"
                  f"Recent trend analysis suggests a weakening in the stock's performance, potentially leading to "
                  f"further declines in price.\n"
                  "Technical indicators such as the MA100 and MA200 suggest a bearish trend, reinforcing the model's "
                  "recommendation to sell.\n"
                  "Selling now could help mitigate potential losses, but it's important to also consider factors like "
                  "company-specific news, sector performance, and broader market conditions before making a final decision.")
else:
    decision = "Hold"
    explanation = (f"The model's predicted price change is within the threshold, indicating no significant movement in the near term ({price_diff_percentage:.2%} difference).\n"
                  "The recent trend shows a stable performance with minimal volatility, as reflected in the technical "
                  "indicators such as the MA50 and MA100.\n"
                  "The current price stability suggests that holding your position could be prudent until clearer "
                  "signals emerge.\n"
                  "It's advisable to keep an eye on market conditions and stay informed about any developments that could "
                  "influence the stock's future performance.")

# Display the decision and explanation
st.subheader(f"Recommendation: {decision}")
st.write(explanation)

Overwriting stock_predictor.py
